# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
from pprint import pprint

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,port-aux-francais,-49.3500,70.2167,5.03,95,85,11.95,TF,1699459098
1,1,sao joao da barra,-21.6403,-41.0511,28.05,57,0,5.29,BR,1699459106
2,2,mokhsogollokh,61.4681,128.9203,-24.20,99,85,1.07,RU,1699459106
3,3,nykoping,58.7530,17.0079,6.92,100,75,2.06,SE,1699459106
4,4,rosarito,32.3333,-117.0333,16.25,66,28,1.74,MX,1699459106


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
# Configure the map plot
map_plot_1 = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    size = "Humidity",
    scale = .5,
    color = "City"
)

# Display the map
map_plot_1

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [4]:
# Narrow down cities that fit criteria and drop any results with null values
ngt3_df = city_data_df[city_data_df['Max Temp']>20]
ngt2_df = ngt3_df[ngt3_df['Humidity']<80]
ngt1_df = ngt2_df[ngt2_df['Cloudiness']<30]
ngt_df= ngt1_df[ngt1_df['Wind Speed']<5]


# Drop any rows with null values
ngt_df.dropna()
ngt_df.dropna(subset=['Country'])

# Display sample data
ngt_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
57,57,khark,29.2614,50.3306,27.01,57,0,4.12,IR,1699459120
63,63,east london,-33.0153,27.9116,20.52,64,0,3.09,ZA,1699459121
73,73,mangrol,21.1167,70.1167,27.79,65,1,0.84,IN,1699459123
79,79,opuwo,-18.0607,13.8400,35.82,8,0,0.88,NaN,1699459125
87,87,alta gracia,-31.6529,-64.4283,28.64,12,6,4.45,AR,1699459126
...,...,...,...,...,...,...,...,...,...,...
549,549,bantva,21.4864,70.0689,27.08,54,9,2.90,IN,1699459295
560,560,hafar al-batin,28.4328,45.9708,27.32,22,0,2.57,SA,1699459298
569,569,nabinagar,23.8917,90.9733,23.07,73,0,2.29,BD,1699459300
573,573,sao vicente,-23.9631,-46.3919,36.26,65,0,3.60,BR,1699459302


### Step 3: Create a new DataFrame called `hotel_df`.

In [5]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ngt_df[['City','Country', 'Lat', 'Lng', 'Humidity']].copy()
hotel_df
# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = ''

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
57,khark,IR,29.2614,50.3306,57,
63,east london,ZA,-33.0153,27.9116,64,
73,mangrol,IN,21.1167,70.1167,65,
79,opuwo,NaN,-18.0607,13.8400,8,
87,alta gracia,AR,-31.6529,-64.4283,12,
...,...,...,...,...,...,...
549,bantva,IN,21.4864,70.0689,54,
560,hafar al-batin,SA,28.4328,45.9708,22,
569,nabinagar,BD,23.8917,90.9733,73,
573,sao vicente,BR,-23.9631,-46.3919,65,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [9]:
# Set parameters to search for a hotel

radius = 10000
params = { 
   'apiKey':geoapify_key
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = hotel_df.loc[index,'Lat']
    longitude = hotel_df.loc[index,'Lng']
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f'circle:{longitude},{latitude},{radius}'
    params["bias"] = f'proximity:{longitude},{latitude}'
    params['categories'] = 'accommodation.hotel'
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places?"

    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    #Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["address_line1"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
khark - nearest hotel: No hotel found
east london - nearest hotel: Inverleith Terrace
mangrol - nearest hotel: No hotel found
opuwo - nearest hotel: Ohakane Lodge
alta gracia - nearest hotel: The Ritz-Carlton
saipan - nearest hotel: Chalan Kanoa Beach Hotel
pangkalpinang - nearest hotel: Renz Hotel
san andres - nearest hotel: Hotel Decameron Isleño
al kharijah - nearest hotel: Kharga Hotel
tres lagoas - nearest hotel: Hotel Sul
port alfred - nearest hotel: Dogs O'Devon
el bauga - nearest hotel: No hotel found
benguela - nearest hotel: Hotel Moibela
awjilah - nearest hotel: Awjilah Ajdabyia Road
san julian - nearest hotel: No hotel found
okakarara - nearest hotel: No hotel found
al kharj - nearest hotel: شقق فندقية
shankargarh - nearest hotel: No hotel found
mukalla - nearest hotel: Al-Jabri Hotel
fayetteville - nearest hotel: Regency Inn
illapel - nearest hotel: Domingo Ortiz de Rosas
jiwani - nearest hotel: No hotel found
goure - nearest hotel: No hotel found
jah

,City,Country,Lat,Lng,Humidity,Hotel Name
57,khark,IR,29.2614,50.3306,57,No hotel found
63,east london,ZA,-33.0153,27.9116,64,Inverleith Terrace
73,mangrol,IN,21.1167,70.1167,65,No hotel found
79,opuwo,NaN,-18.0607,13.8400,8,Ohakane Lodge
87,alta gracia,AR,-31.6529,-64.4283,12,The Ritz-Carlton
...,...,...,...,...,...,...
549,bantva,IN,21.4864,70.0689,54,No hotel found
560,hafar al-batin,SA,28.4328,45.9708,22,فندق الفرحان
569,nabinagar,BD,23.8917,90.9733,73,No hotel found
573,sao vicente,BR,-23.9631,-46.3919,65,Pousada Vitória


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [14]:
# Configure the map plot
map_plot_2 = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    size = "Humidity",
    scale = .5,
    color = "City",
    hover_cols=["Hotel Name", 'City', 'Country', 'Lng', 'Lat', 'Humidity']
)


# Display the map
map_plot_2

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)